# Lista 4: Q&A

Pobierando:

In [ ]:
!pip install langchain sentence-transformers faiss-cpu pypdf transformers torch langchain-community

In [ ]:
!pip install pypdfium2

In [46]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import FAISS
from langchain import PromptTemplate
from transformers import pipeline
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline


In [14]:
from langchain.document_loaders import PyPDFium2Loader

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Ładowanie artykułów

Tworzymy funkcję do ładowania pdfów. Wybrałem 4 artykuły o wpływie AI na edukację. Artykuły te są w folderze na dysku. Pobieramy je, dzielimy na chunki po 300 znaków gdzie 100 jest z sąsiedniego chunka dla zachowania kontekstu

In [20]:
def load_pdf(article,size,overlap):
    loader = PyPDFium2Loader(article)
    pages = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=size,               # maksymalny rozmiar chunk'a
        chunk_overlap=overlap,    # ile znaków nakłada się na sąsiednie chunki, żeby nie zgubić kontekstu
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )

    docs = text_splitter.split_documents(pages)
    return docs


drive = "/content/drive/MyDrive/ml"

# Pobieramy wszystkie pliki PDF w folderze
articles = [os.path.join(drive, f) for f in os.listdir(drive)]

docs = []
for i in articles:
    doc = load_pdf(i, 300, 100)
    docs.extend(doc)

# nie będę pokazywać całości bo jest tego sporo ale tak to wygląda
for i, doc in enumerate(docs[100:105]):
    print(f"[{ i }] {doc}\n")


[0] page_content='teaching jobs is a priority, and via automated assistants or other tools, AI may provide teachers 
greater support. AI may also enable teachers to extend the support they offer to individual 
students when they run out of time. Developing resources that are responsive to the knowledge' metadata={'producer': '', 'creator': 'Microsoft Word', 'creationdate': '2023-05-02T18:36:15+00:00', 'title': 'Artificial Intelligence and the Future of Teaching and Learning (PDF)', 'author': 'U.S. Department of Education', 'subject': '', 'keywords': '', 'moddate': '2023-05-22T15:08:44-04:00', 'source': '/content/drive/MyDrive/ml/ai-report.pdf', 'total_pages': 71, 'page': 5}

[1] page_content='students when they run out of time. Developing resources that are responsive to the knowledge 
and experiences students bring to their learning—their community and cultural assets—is a 
priority, and AI may enable greater customizability of curricular resources to meet local needs.' metadata={'pro

## Tworzenie embeddingu i bazy danych:

Ze strony hugging face podanej przy zadaniu znalazłem model do embedingu "sentence-transformers/all-MiniLM-L6-v2" którego zamierzam użyć. Naszym modelem językowym będzie "google/flan-t5-base" od googla

In [ ]:
# tak jak w cheat sheet'cie
embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
database = FAISS.from_documents(docs, embeddings)
database.save_local("baza danych lista 4")
database = FAISS.load_local("baza danych lista 4", embeddings, allow_dangerous_deserialization=True)

## Model:

Od razu do modelu zaimplementowałem pokazywanie źródeł oraz pozbyłem się cyberpsychozy dodatkowym promptem.

In [45]:
model_name = "google/flan-t5-base"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    do_sample=False       # deterministyczne odpowiedzi
)

llm = HuggingFacePipeline(pipeline=pipe)

# robimy promt który zredukuje cyberpsychozę. mówi on modelowi aby nie bajerował tylko od razu pisał że nie wie
template = """Answer the question ONLY based on the context provided.
If the answer is not in the context, respond with: "I don't know".


Contextt:
{context}

Question: {question}

Answer:"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=template
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)

while not end:
    q = str(input("Ask a question: "))
    if q.lower() != "exit":
        print(f"User: {q}")

        result = qa({"query": q})

        print(f"Answer: {result['result']}\n")

        # jeżeli istnieje źródło (a istnieje bo inaczej jest i dont know) to je pokazujemy pod odpowiedzią
        if result['result'] != "I don't know":
          print("Sources (fragments from documents):")
          for doc in result['source_documents']:
              # dla zachowania przejrzystości dam tylko początek
              snippet = doc.page_content[:200].replace("\n", " ")
              print(f"- {snippet}...")
        print("\n----------------\n")

    else:
        end = True

Device set to use cpu


Ask a question: In wchich fields AI can support teachers
User: In wchich fields AI can support teachers
Answer: AI assistants to reduce routine teaching burdens; AI that provides teachers with recommendations teacher to a facilitator

Sources (fragments from documents):
- people choose the teaching profession and ensure they can do the work that matters. This  section discusses examples of AI supporting teachers and teaching including these concepts: AI  assistants to ...
- teacher to a facilitator (Holmes and Tuomi, 2022). Teachers can integrate AI lessons as supplementary materials to  assist weak students and provide hands-on experiences in the form of human interacti...
- teaching jobs is a priority, and via automated assistants or other tools, AI may provide teachers  greater support. AI may also enable teachers to extend the support they offer to individual  students...
- argued that there are some subtle challenges related to the use of AI in the education field.  6. Conclusion 

Odpowiedzi w miarę logiczne, wydaję mi się że model odpowiada poprawnie, przytoczone fragmenty źródeł też by się zgadzały. zgodnie z założeniami model nie wie o istnieniu człowieka wielu talentów (a szkoda)